In [239]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split

#Importando o modelo machine learnig
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import pickle



In [240]:
# importar csv
data = pd.read_csv("nge_dados_input_final.csv")
#data_semNF = pd.read_csv("nge_dados_input_semNF.csv") # jobs sem nota está emoutro método

In [241]:
data.dtypes

id                 int64
pag_final          int64
qtd_pg_word        int64
qtd_carc           int64
qtd_tabela         int64
qtd_image          int64
qtd_estilos      float64
Data              object
tamanho_livro      int64
dtype: object

In [242]:
data.head(12)

,id,pag_final,qtd_pg_word,qtd_carc,qtd_tabela,qtd_image,qtd_estilos,Data,tamanho_livro
0,224323,140,87,116142,505,742,60.0,2018-09-20,2
1,224325,122,82,108307,76,94,69.0,2018-09-20,2
2,225392,52,30,41144,28,24,23.0,2018-11-07,1
3,225392,52,42,62913,432,696,21.0,2018-11-05,1
4,225531,32,36,37548,270,456,17.0,2018-11-07,0
5,228711,48,77,101967,317,517,17.0,2019-03-28,0
6,226842,88,46,96474,99,114,37.0,2019-01-15,1
7,227130,66,66,112888,604,872,23.0,2020-11-09,1
8,228307,82,62,43155,40,73,25.0,2019-02-26,1
9,228354,92,76,129122,14,10,26.0,2019-02-27,1


In [243]:
data['qtd_pg_word'].unique()

array([ 87,  82,  30,  42,  36,  77,  46,  66,  62,  76,  80,  13,  34,
        52,  18,  78,  50,  47,  54,  88,  91,  38, 214,  22,  26,  92,
        14,  40,  81,  63, 216,  24,  33, 195,  29,  60,  70,  72, 107,
        85,  68,  41, 112,  45,  19,  51,  83,  31,  56,  28,  21,  12,
        17, 131,  67, 207,  43,  73,  86,  90,  49, 106,  39,  59,  48,
        64,  23,  44,  20, 142,  11, 424, 374,  32,  57,  37])

In [244]:
# tentanto estatificar o dataframe
print(len(data[data["pag_final"]>149]))
print(len(data[(data["pag_final"]>59) & (data["pag_final"]<150)]))
print(len(data[data["pag_final"]<60]))

9
67
63


In [245]:
print(len(data[data["tamanho_livro"]==2]))
print(len(data[data["tamanho_livro"]==1]))
print(len(data[data["tamanho_livro"]==0]))

16
95
28


# fazer um critério de seleção de dados:
### se o dado colhido na tabela (qtd_pag_word) tiver diferente da raspagem do word (qtd_pg_word) mais de 5 pontos deve retirar do arquivo de entrada. 
###  - incluir a coluna raspagem do word (qtd_pg_word)  para treinamento

In [246]:
data['id'].count()

139

In [247]:
data.describe()

,id,pag_final,qtd_pg_word,qtd_carc,qtd_tabela,qtd_image,qtd_estilos,tamanho_livro
count,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000
mean,219735.294964,76.733813,59.359712,90749.014388,393.913669,432.057554,27.266187,0.913669
std,54569.337684,58.271301,55.227068,101685.944703,282.048549,239.173631,14.970297,0.557973
min,0.000000,1.000000,11.000000,12032.000000,2.000000,2.000000,3.000000,0.000000
25%,230981.500000,42.500000,29.500000,32448.000000,152.000000,262.000000,18.500000,1.000000
50%,233887.000000,66.000000,46.000000,70528.000000,351.000000,438.000000,23.000000,1.000000
75%,235519.500000,91.000000,70.000000,101969.000000,570.000000,634.000000,29.000000,1.000000
max,236123.000000,386.000000,424.000000,660609.000000,929.000000,872.000000,127.000000,2.000000


In [248]:
#
# normalizar a quantidade de caracteres
#
data.loc[:,'qtd_carc_normal'] = pd.Series((data.qtd_carc-data.qtd_carc.min())/(data.qtd_carc.max()-data.qtd_carc.min()))

In [249]:
#########################################################
# INÍCIO DA PARTE DE MACHINE LEARNIG  \
# RANDOM FOREST
########################################################

In [250]:
# instalar biblioteca scikit-learn
#! pip install -U scikit-learn

In [251]:
data.columns

Index(['id', 'pag_final', 'qtd_pg_word', 'qtd_carc', 'qtd_tabela', 'qtd_image',
       'qtd_estilos', 'Data', 'tamanho_livro', 'qtd_carc_normal'],
      dtype='object')

In [252]:
colunas_X= ['qtd_pg_word','qtd_tabela','qtd_image', 'qtd_estilos', 'qtd_carc','tamanho_livro']

# ficou de fora: 'qtd_estilos', 'qtd_carc_normal']

colunas_y=['pag_final']
X = data[colunas_X]
y = data[colunas_y]

In [253]:
X.columns

Index(['qtd_pg_word', 'qtd_tabela', 'qtd_image', 'qtd_estilos', 'qtd_carc',
       'tamanho_livro'],
      dtype='object')

In [254]:
# método train_test_split, Scikit-Learn 
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X, y, test_size=0.20, random_state=0,stratify=X['tamanho_livro'])

In [255]:
X_train_f.shape, X_test_f.shape, y_train_f.shape, y_test_f.shape

((111, 6), (28, 6), (111, 1), (28, 1))

In [256]:
y_test_f.head()

,pag_final
75,46
98,110
44,46
29,46
73,86


In [257]:
X_train_f

,qtd_pg_word,qtd_tabela,qtd_image,qtd_estilos,qtd_carc,tamanho_livro
53,41,571,852,30.0,42669,1
104,40,39,25,22.0,42151,2
86,49,895,682,20.0,103663,1
87,70,220,438,17.0,95656,2
18,47,85,111,28.0,87338,1
...,...,...,...,...,...,...
56,45,315,495,40.0,70054,1
68,21,738,559,12.0,29631,0
128,30,482,419,43.0,66895,1
77,207,763,568,24.0,571640,1


In [258]:
X_train_f.dtypes

qtd_pg_word        int64
qtd_tabela         int64
qtd_image          int64
qtd_estilos      float64
qtd_carc           int64
tamanho_livro      int64
dtype: object

In [259]:
y_train_f

,pag_final
53,98
104,162
86,68
87,296
18,28
...,...
56,104
68,40
128,38
77,20


In [260]:
print(X_train_f.shape,y_train_f.shape)

(111, 6) (111, 1)


In [261]:
print(X_train_f.shape,y_train_f.values.ravel().shape)

(111, 6) (111,)


In [262]:
modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
modelo = modelo.fit(X_train_f, y_train_f.values.ravel())


In [263]:
p = modelo.predict(X_test_f)

In [264]:
p

array([ 38,  78,  52, 122,  68,  96, 120, 122,  57,  68,  44,  68,  68,
        43,  76,  66, 126,  96,  54,  48,  48,  68,  48, 340, 168,  48,
        28,  40])

In [265]:
print(p.shape,y_test_f.shape)

(28,) (28, 1)


In [266]:
y_test_f

,pag_final
75,46
98,110
44,46
29,46
73,86
121,66
102,128
15,46
97,57
21,84


In [267]:
# Calculando RMSE utilizando scikit-learn
print('RMSE: ',np.sqrt(metrics.mean_squared_error(y_test_f,p)))
# RMSE:  78.92781512242689
# RMSE:  78.58244078673046
# RMSE:  19.371957239079567
# RMSE:  53.665631459994955
# RMSE:  26.642413213111567 sem a feature quantidade de estilo
# RMSE:  28.926238482168273   ['qtd_pag_word', 'qtd_carc','qtd_tabela','qtd_image']
# RMSE:  9.302980949429841 depois de configurar split por estratificação
# RMSE:  14.142135623730951
# RMSE:  18.138357147217054

RMSE:  53.89970845402201


In [268]:
# Save Model Using Pickle
# save the model to disk

filename = 'finalized_model-forest_final.sav'
pickle.dump(modelo, open(filename, 'wb'))
 

In [269]:
##############################################################################################################

In [270]:
#########################################################
# FIM RANDOM FOREST
########################################################

In [271]:
##############################################################################################################

In [272]:
#########################################################
# INÍCIO DA PARTE DE MACHINE LEARNIG  \
# LINEAR REGRESSION
########################################################

In [273]:

colunas_X= ['id', 'qtd_pg_word','qtd_carc','qtd_tabela','qtd_image',"qtd_estilos",'qtd_carc','tamanho_livro']


colunas_y=['pag_final']
X = data[colunas_X]
y = data[colunas_y]

In [274]:
# método train_test_split, Scikit-Learn 
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.20,random_state=0,stratify=X['tamanho_livro']) # feito com 0,10 e agora com 0,20

In [275]:
####
####   selecionar a FEATURES para MODELO
####
# mantendo algumas features para compara resultados no final no arquivo X_train_ref
X_test_ref = X_test.copy()
X_train_ref= X_train.copy()

X_test = X_test_ref[['qtd_pg_word', 'qtd_tabela','qtd_image', 'qtd_carc','tamanho_livro']]

X_train = X_train_ref[['qtd_pg_word', 'qtd_tabela','qtd_image', 'qtd_carc','tamanho_livro']]


In [276]:
X_test_ref.columns

Index(['id', 'qtd_pg_word', 'qtd_carc', 'qtd_tabela', 'qtd_image',
       'qtd_estilos', 'qtd_carc', 'tamanho_livro'],
      dtype='object')

In [277]:
X_train

,qtd_pg_word,qtd_tabela,qtd_image,qtd_carc,qtd_carc,tamanho_livro
53,41,571,852,42669,42669,1
104,40,39,25,42151,42151,2
86,49,895,682,103663,103663,1
87,70,220,438,95656,95656,2
18,47,85,111,87338,87338,1
...,...,...,...,...,...,...
56,45,315,495,70054,70054,1
68,21,738,559,29631,29631,0
128,30,482,419,66895,66895,1
77,207,763,568,571640,571640,1


In [278]:
y_train

,pag_final
53,98
104,162
86,68
87,296
18,28
...,...
56,104
68,40
128,38
77,20


In [279]:
##### machine learnig

#Instanciando o estimador
linreg = LinearRegression()

#Ajustando o modelo ao conjunto de treino
linreg.fit(X_train, y_train)

#Interpretando os coeficientes do modelo
#Visualizando a interseção e os coeficientes 
print(linreg.intercept_)
print('coeficientes encontrados: ',linreg.coef_) # para visualizar os coeficientes encontrados



[-1.98021585]
coeficientes encontrados:  [[ 3.65194773e-01 -5.66773734e-02  7.92144091e-02 -9.32712157e-05
  -9.32713670e-05  6.69966106e+01]]


In [280]:
# para visualizar os coeficientes encontrados
# para visualizar o R²
# linreg.score()

In [281]:
colunas_X

['id',
 'qtd_pg_word',
 'qtd_carc',
 'qtd_tabela',
 'qtd_image',
 'qtd_estilos',
 'qtd_carc',
 'tamanho_livro']

In [282]:
X_train.columns

Index(['qtd_pg_word', 'qtd_tabela', 'qtd_image', 'qtd_carc', 'qtd_carc',
       'tamanho_livro'],
      dtype='object')

In [283]:
#Atributos e seus coeficientes
list(zip(colunas_X, linreg.coef_))

[('id',
  array([ 3.65194773e-01, -5.66773734e-02,  7.92144091e-02, -9.32712157e-05,
         -9.32713670e-05,  6.69966106e+01]))]

In [284]:
#Fazendo predições no conjunto de teste
y_pred = linreg.predict(X_test)

In [285]:
y_pred

array([[ 74.75577591],
       [ 71.65098031],
       [ 75.66250738],
       [ 70.813169  ],
       [ 71.21967612],
       [ 51.53924703],
       [138.57236351],
       [ 80.54262325],
       [ 12.61868761],
       [ 94.47534085],
       [ 65.3210475 ],
       [ 74.68681566],
       [ 72.10654193],
       [ 68.64115144],
       [101.05862156],
       [ 94.96300064],
       [ 75.91815544],
       [ 74.91867106],
       [111.4313846 ],
       [  6.17979578],
       [  2.76483895],
       [ 78.22440002],
       [ 22.93919264],
       [139.0743258 ],
       [161.02113472],
       [ 24.98137481],
       [ 88.43578853],
       [  5.94343805]])

In [286]:
y_test

,pag_final
75,46
98,110
44,46
29,46
73,86
121,66
102,128
15,46
97,57
21,84


Comparando essas métricas:

- MAE é a mais fácil de entender.
- MSE é mais popular que MAE, porque MSE "pune" erros grandes.
- RMSE é ainda mais popular que MSE, porque RMSE é interpretável na mesma unidade de y.

In [287]:
#Nós precisamos de uma métrica de avaliação para comparar nossas predições com os valores verdadeiros!

# Calculando RMSE utilizando scikit-learn
RMSE_1 = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
print('RMSE: ',RMSE_1)

# incluido feature Quantidade de estilo, mas o resultado piorou. RMSE foi de 11.83 para 12.43
# RMSE:  12.051420343980208
# RMSE:  9.782602553564418 => melhorou quando retirou feature complexidade
# RMSE:  9.16223875809479 => melhorou quando retirou feature quantidade de estilos
# RMSE:  4.5881270718708045 => melhorou MUITO depois de verificar os arquivo word que estavam errados
# RMSE:  4.303364983662433 => melhorou quando alterei arquivos do WORD

# Campos em análise [['complexidade', 'qtd_pag_word', 'qtd_carc','qtd_tabela','qtd_image',"qtd_estilos",'qtd_carc_normal']]
#RMSE:  12.051420343982636
# RMSE:  16.219875583371206
#RMSE:  14.883956821122942

RMSE:  46.66384582961599


In [288]:
X_test_ref.columns

Index(['id', 'qtd_pg_word', 'qtd_carc', 'qtd_tabela', 'qtd_image',
       'qtd_estilos', 'qtd_carc', 'tamanho_livro'],
      dtype='object')

In [289]:
X_test.columns

Index(['qtd_pg_word', 'qtd_tabela', 'qtd_image', 'qtd_carc', 'qtd_carc',
       'tamanho_livro'],
      dtype='object')

In [290]:
#Como interpretar esse valor?
# numero menor de RMSE é melhor
# RMSE:  11.831745496309976
# RMSE:  12.43000991830557
# RMSE:  14.428565435305106


In [291]:
# Save Model Using Pickle
# save the model to disk

filename = 'finalized_model-linreg_final.sav'
pickle.dump(linreg, open(filename, 'wb'))
 


In [292]:
#########################################################
# FIM  LINEAR REGRESSION
########################################################

In [293]:
################################################################################################################
#########################################################
# LOGISTIC REGRESSION
########################################################

In [294]:

colunas_X= ['qtd_pg_word', 'qtd_tabela','qtd_image',"qtd_estilos", 'qtd_carc','tamanho_livro']


colunas_y=['pag_final']
X = data[colunas_X]
y = data[colunas_y]

In [295]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.20,random_state=0,stratify=X['tamanho_livro'])

In [296]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
##### machine learnig
#X, y = load_iris(return_X_y=True)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)

# modelo = LogisticRegression()
# modelo.fit(X_treino, y_treino)
# p = modelo.predict(X_valid)



print(clf.predict(X_test.fillna(-1)))
print(clf.predict_proba(X_test.fillna(-1)))
print(clf.score(X_train, y_train))


[48 42 48 48 48 48 42 48 48 48 48 48 48 42 48 48 48 48 48 68 48 48 48 68
 48 48 48 48]
[[1.12472148e-56 2.72205599e-02 5.04620596e-03 ... 1.34508111e-02
  9.90867879e-04 1.34508111e-02]
 [4.16613662e-13 2.22461185e-02 1.58496274e-02 ... 1.92099067e-02
  1.12925769e-02 1.92099067e-02]
 [6.67790779e-19 2.25517734e-02 1.37113994e-02 ... 1.84809824e-02
  8.65332815e-03 1.84809824e-02]
 ...
 [3.71230627e-19 1.80078341e-02 1.20463884e-02 ... 1.64861257e-02
  9.72476223e-03 1.64861257e-02]
 [4.58503150e-10 1.68925509e-02 1.48297685e-02 ... 1.72183199e-02
  1.53179066e-02 1.72183199e-02]
 [8.00292867e-10 1.93154414e-02 1.60520992e-02 ... 1.83708709e-02
  1.43005125e-02 1.83708709e-02]]
0.09009009009009009


/Users/wpessoa/miniconda3/envs/py3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/wpessoa/miniconda3/envs/py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [297]:
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test.fillna(-1))
print(y_pred)
print(y_test.T)

[48 42 48 48 48 48 42 48 48 48 48 48 48 42 48 48 48 48 48 68 48 48 48 68
 48 48 48 48]
           75   98   44   29   73   121  102  15   97   21   ...  49   115  \
pag_final   46  110   46   46   86   66  128   46   57   84  ...   66   40   

           66   85   54   69   27   4    57   14   
pag_final   48   88   40  196  340   32   96   40  

[1 rows x 28 columns]


/Users/wpessoa/miniconda3/envs/py3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/wpessoa/miniconda3/envs/py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [298]:
# Calculando RMSE utilizando scikit-learn
print('RMSE: ',np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
# RMSE:  26.17424132789543 => Features ['qtd_pag_word', 'qtd_tabela','qtd_image',"qtd_estilos", 'qtd_carc_normal']

# quantidade de estilo é muito relevante nesse modelo, sem essa feature RMSE:  106.26211160915429
# RMSE:  92.03951720469163 Para essa features ['qtd_pag_word', 'qtd_carc','qtd_tabela','qtd_image']
# RMSE:  19.521549863955705


RMSE:  66.88289339264135


In [299]:
# Save Model Using Pickle
# save the model to disk

filename = 'finalized_model-log_final.sav'
pickle.dump(modelo, open(filename, 'wb'))
 

# Modelos treinados salvo em:
- finalized_model-forest3a.sav
- finalized_model-linreg.sav
- finalized_model-log.sav

In [ ]:
#########################################################
# FIM  LOGISTIC REGRESSION
########################################################

In [ ]:
#
# 
################################################################################################################